In [13]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import numpy as np
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import SGDClassifier
import random

In [3]:
#Load Data
features = [i.replace(' ', '_') for i in load_breast_cancer().feature_names.tolist()]

breast_cancer_df = pd.DataFrame(load_breast_cancer().data,columns=features)
target_df = pd.DataFrame(load_breast_cancer().target, columns=['y'])

df = pd.concat([target_df,breast_cancer_df],axis=1)

In [30]:
#Create small data set and rare event data set
num_successes_for_rare = int(((((1-df.y.mean())*df.shape[0])/.95)-((1-df.y.mean())*df.shape[0]))//1)
rare_inds = sorted(list(df[df.y==0].index) + random.sample(list(df[df.y==1].index),num_successes_for_rare))
small_inds = random.sample(sorted(list(df.index)),50)
rare_df = df.iloc[rare_inds,:]
small_df = df.iloc[small_inds,:]

In [88]:
def DAP(df,y_var_name):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    return f_df, weights


        

In [90]:
all_aug, all_weights = DAP(df,'y')
small_aug, small_weights = DAP(small_df,'y')
rare_aug, rare_weights = DAP(rare_df,'y')

/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
#Convert data to R
with localconverter(ro.default_converter + pandas2ri.converter):
    #for firth
    firth_all_r = ro.conversion.py2ri(df)
    firth_small_r = ro.conversion.py2ri(small_df)
    firth_rare_r = ro.conversion.py2ri(rare_df)
    #for 
    flog_all_r = ro.conversion.py2ri(all_aug)
    all_weights_r = ro.vectors.FloatVector(all_weights)
    flog_small_r = ro.conversion.py2ri(small_aug)
    small_weights_r = ro.vectors.FloatVector(small_weights)
    flog_rare_r = ro.conversion.py2ri(rare_aug)
    rare_weights_r = ro.vectors.FloatVector(rare_weights)

In [10]:
base = importr('base')
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
brglm = importr('brglm',robject_translations=d)

In [ ]:
brglm.brglm()